### CREATING THE DATABASE AND TABLES

In [1]:
!pip install country_converter

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import country_converter as coco

import seaborn as sns
import missingno as msno
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot
from PIL import Image
import requests

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import warnings
warnings.filterwarnings('ignore')

In [2]:
books = pd.read_csv("./books.csv")
ratings = pd.read_csv("./rating.csv")
users = pd.read_csv("./users.csv")

In [3]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Genre'],
      dtype='object')

In [4]:
ratings.columns

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')

In [5]:
users.columns

Index(['User-ID', 'Location', 'Age'], dtype='object')

In [6]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Genre                  0
dtype: int64

In [7]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [8]:
users.isnull().sum()

User-ID       0
Location      0
Age         111
dtype: int64

In [9]:
import pandas as pd
# Replace null values in 'Age' with the median age
users['Age'].fillna(users['Age'].median(), inplace=True)

# Verify if null values are replaced
print(users.isnull().sum())


User-ID     0
Location    0
Age         0
dtype: int64


### creating database and tables

In [10]:
import mysql.connector
from mysql.connector import Error
# Database connection details
HOST = "localhost"
USER = "root"
PASSWORD = "root"
try:
    # Connect to MySQL server
    connection = mysql.connector.connect(
        host=HOST,
        user=USER,
        password=PASSWORD
    )
    if connection.is_connected():
        print(":white_check_mark: Connected to MySQL Server")
        # Create a cursor object
        cursor = connection.cursor()
        # Step 1: Create the database
        cursor.execute("CREATE DATABASE IF NOT EXISTS genrebookrecommendation")
        print(":white_check_mark: Database 'genrebookrecommendation' created successfully")
        # Switch to the new database
        cursor.execute("USE genrebookrecommendation")
        # Step 2: Create Books table
        create_books_table = """
        CREATE TABLE IF NOT EXISTS Books (
            ISBN VARCHAR(20) PRIMARY KEY,
            Book_Title VARCHAR(255) NOT NULL,
            Book_Author VARCHAR(255) NOT NULL,
            Year_Of_Publication INT NOT NULL,
            Publisher VARCHAR(255) NOT NULL,
            Genre VARCHAR(100) NOT NULL
        )
        """
        cursor.execute(create_books_table)
        print(":white_check_mark: Table 'Books' created successfully")
        # Step 3: Create Users table
        create_users_table = """
        CREATE TABLE IF NOT EXISTS Users (
            User_ID INT PRIMARY KEY,
            Location VARCHAR(255) NOT NULL,
            Age INT NOT NULL
        )
        """
        cursor.execute(create_users_table)
        print(":white_check_mark: Table 'Users' created successfully")
        # Step 4: Create Ratings table (with Composite Primary Key and Foreign Keys)
        create_ratings_table = """
        CREATE TABLE IF NOT EXISTS Ratings (
            User_ID INT NOT NULL,
            ISBN VARCHAR(20) NOT NULL,
            Book_Rating INT CHECK (Book_Rating BETWEEN 1 AND 10),
            PRIMARY KEY (User_ID, ISBN),  -- Composite primary key
            FOREIGN KEY (ISBN) REFERENCES Books(ISBN) ON DELETE CASCADE
        )
        """
        cursor.execute(create_ratings_table)
        print(":white_check_mark: Table 'Ratings' created successfully")
except Error as e:
    print(f":x: Error: {e}")
finally:
    # Close the connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print(":white_check_mark: MySQL connection is closed")

:white_check_mark: Connected to MySQL Server
:white_check_mark: Database 'genrebookrecommendation' created successfully
:white_check_mark: Table 'Books' created successfully
:white_check_mark: Table 'Users' created successfully
:white_check_mark: Table 'Ratings' created successfully
:white_check_mark: MySQL connection is closed


### instreating the data into database

In [11]:
import mysql.connector
import pandas as pd
# Database connection details
HOST = "localhost"
USER = "root"
PASSWORD = "root"
DATABASE = "genrebookrecommendation"
# Connect to the MySQL database
try:
    connection = mysql.connector.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DATABASE
    )
    if connection.is_connected():
        print(":white_check_mark: Connected to MySQL Database")
    cursor = connection.cursor()
    # Function to insert data into a table
    def insert_data_from_csv(table_name, csv_file, columns):
        # Load the CSV file with explicit data types and strip column names
        data = pd.read_csv(csv_file, dtype=str).rename(columns=lambda x: x.strip())
        # Handle missing values
        if 'Age' in data.columns:
            data['Age'] = data['Age'].fillna(25).astype(int)  # Replace NaN Age with 25
        if 'Book_Rating' in data.columns:
            data['Book_Rating'] = pd.to_numeric(data['Book_Rating'], errors='coerce').fillna(5).astype(int)  # Convert and fill NaN
            data['Book_Rating'] = data['Book_Rating'].clip(lower=1, upper=10)  # Ensure within 1-10
        data = data.fillna('NULL')  # Replace remaining NaN values
        # Prepare the INSERT query with the IGNORE keyword to skip duplicates
        placeholders = ", ".join(["%s"] * len(columns))
        query = f"INSERT IGNORE INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})"
        # Insert each row into the table
        for _, row in data.iterrows():
            cursor.execute(query, tuple(row))
        connection.commit()
        print(f":white_check_mark: Data inserted into {table_name} successfully")
    # Insert Books data
    insert_data_from_csv(
        table_name="books",
        csv_file="./books.csv",
        columns=['ISBN', 'Book_Title', 'Book_Author', 'Year_Of_Publication', 'Publisher', 'Genre']
    )
    # Insert Users data
    insert_data_from_csv(
        table_name="users",
        csv_file="./users.csv",
        columns=['User_ID', 'Location', 'Age']
    )
    # Insert Ratings data
    insert_data_from_csv(
        table_name="ratings",
        csv_file="./rating.csv",
        columns=['User_ID', 'ISBN', 'Book_Rating']
    )
except mysql.connector.Error as e:
    print(f":x: Error: {e}")
finally:
    # Close the connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print(":white_check_mark: MySQL connection is closed")

:white_check_mark: Connected to MySQL Database
:white_check_mark: Data inserted into books successfully
:white_check_mark: Data inserted into users successfully
:white_check_mark: Data inserted into ratings successfully
:white_check_mark: MySQL connection is closed


### extracting the data from sql

In [12]:
import mysql.connector
import pandas as pd
# Database connection details
HOST = "localhost"
USER = "root"
PASSWORD = "root"
DATABASE = "genrebookrecommendation"  # Your database name
# Connect to the MySQL database
try:
    connection = mysql.connector.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DATABASE
    )
    if connection.is_connected():
        print(":white_check_mark: Connected to MySQL Database")
    # Function to fetch data from a table and return it as a Pandas DataFrame
    def fetch_data(table_name):
        query = f"SELECT * FROM {table_name}"  # Fetch all data
        df = pd.read_sql(query, connection)
        return df
    # Fetching data from tables
    books_df = fetch_data("books")
    users_df = fetch_data("users")
    ratings_df = fetch_data("ratings")
    # Display sample data
    print("\n:books: Books Table Sample:")
    display(books_df.head())
    print("\n:bust_in_silhouette: Users Table Sample:")
    display(users_df.head())
    print("\n:star: Ratings Table Sample:")
    display(ratings_df.head())
except mysql.connector.Error as e:
    print(f":x: Error: {e}")
finally:
    # Close the connection
    if connection.is_connected():
        connection.close()
        print(":white_check_mark: MySQL connection is closed")

:white_check_mark: Connected to MySQL Database

:books: Books Table Sample:


,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher,Genre
0,006015781X,Why Do Clocks Run Clockwise? and Other Imponde...,David Feldman,1987,Harpercollins,Travel
1,006017143X,The Night Listener,Armistead Maupin,2000,HarperCollins Publishers,Religion
2,006052779X,Charlotte's Web Book and Charm (Charming Class...,E. B. White,2003,HarperFestival,Health
3,006101351X,The Perfect Storm : A True Story of Men Agains...,Sebastian Junger,1998,HarperTorch,History
4,006102063X,Moving Pictures (Discworld Novels (Paperback)),Terry Pratchett,2002,HarperTorch,Travel



:bust_in_silhouette: Users Table Sample:


,User_ID,Location,Age
0,2,"stockton, california, usa",18
1,8,"timmins, ontario, canada",25
2,9,"germantown, tennessee, usa",25
3,10,"albacete, wisconsin, spain",26
4,12,"fort bragg, california, usa",25



:star: Ratings Table Sample:


,User_ID,ISBN,Book_Rating
0,8,074322678X,5
1,8,1552041778,5
2,8,1567407781,6
3,8,1575663937,6
4,8,1881320189,7


:white_check_mark: MySQL connection is closed


### content filtering 

In [39]:
import mysql.connector
import pandas as pd
import panel as pn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
pn.extension()
def connect_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="genrebookrecommendation"
    )
connection = connect_db()
cursor = connection.cursor()
print(":white_check_mark: Connected to MySQL Database")
def fetch_data(query):
    return pd.read_sql(query, connection)
books_df = fetch_data("SELECT ISBN, Book_Title, Book_Author, Publisher, Genre FROM books")
users_df = fetch_data("SELECT User_ID FROM users")
ratings_df = fetch_data("SELECT User_ID, ISBN FROM ratings")
ratings_books_df = ratings_df.merge(books_df, on='ISBN', how='inner')
# Create user profiles
user_profiles = ratings_books_df.groupby('User_ID')['Book_Title'].apply(lambda x: " ".join(x)).reset_index()
user_profiles.rename(columns={'Book_Title': 'user_profile'}, inplace=True)
users_df = users_df.merge(user_profiles, on='User_ID', how='left')
# Create book profiles
books_df['book_profile'] = books_df.apply(
    lambda x: f"{x['Book_Title']} {x['Book_Author']} {x['Publisher']} {x['Genre']}", axis=1
)
# Remove missing profiles
users_df.dropna(subset=['user_profile'], inplace=True)
books_df.dropna(subset=['book_profile'], inplace=True)
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_books = tfidf_vectorizer.fit_transform(books_df['book_profile'])
tfidf_users = tfidf_vectorizer.transform(users_df['user_profile'])
# Compute similarity
cosine_sim = cosine_similarity(tfidf_users, tfidf_books)
user_id_to_index = {int(row['User_ID']): idx for idx, row in users_df.iterrows()}
# Updated recommendation function
def get_recommendations(user_id, genre=None, top_n=5):
    if user_id in user_id_to_index:
        user_idx = user_id_to_index[user_id]
        sim_scores = list(enumerate(cosine_sim[user_idx]))
        sim_scores_df = pd.DataFrame(sim_scores, columns=['Index', 'Similarity'])
        sim_scores_df = sim_scores_df.sort_values(by='Similarity', ascending=False)
        recommended_books = books_df.iloc[sim_scores_df['Index']][['Book_Title', 'Book_Author', 'Genre']]
        if genre:
            recommended_books = recommended_books[recommended_books['Genre'].str.lower() == genre.lower()]
        return recommended_books.head(top_n)
    else:
        # Fixed: New users should get books strictly from the selected genre
        if genre:
            genre_books = books_df[books_df['Genre'].str.lower() == genre.lower()]
            if len(genre_books) < top_n:
                return genre_books  # If not enough books, return available ones
            return genre_books.sample(n=top_n)  # Strict genre filtering
        return books_df.sample(n=top_n)[['Book_Title', 'Book_Author', 'Genre']]
# Panel UI
dropdown = pn.widgets.Select(name='Select User', options=list(user_id_to_index.keys()) + ['New User'])
genre_dropdown = pn.widgets.Select(name='Select Genre', options=['All'] + list(books_df['Genre'].unique()))
user_id_input = pn.widgets.IntInput(name='User ID')
age_input = pn.widgets.IntInput(name='Age')
location_input = pn.widgets.TextInput(name='Location')
save_button = pn.widgets.Button(name='Save New User', button_type='success')
recommendations_output = pn.pane.Markdown()
def update_recommendations(event):
    selected_user = dropdown.value
    selected_genre = genre_dropdown.value if genre_dropdown.value != 'All' else None
    if selected_user == "New User":
        recommendations_output.object = ":bust_in_silhouette: Enter details for new user."
    else:
        rec_books = get_recommendations(selected_user, genre=selected_genre)
        recommendations_output.object = rec_books.to_markdown()
def save_new_user(event):
    user_id = user_id_input.value
    age = age_input.value
    location = location_input.value
    selected_genre = genre_dropdown.value if genre_dropdown.value != 'All' else None
    if not user_id or not age or not location:
        recommendations_output.object = ":x: Please fill in all fields!"
        return
    try:
        connection = connect_db()
        cursor = connection.cursor()
        cursor.execute("SELECT User_ID FROM users WHERE User_ID = %s", (user_id,))
        existing_user = cursor.fetchone()
        if existing_user:
            recommendations_output.object = f":warning: User {user_id} already exists!"
        else:
            cursor.execute("INSERT INTO users (User_ID, Age, Location) VALUES (%s, %s, %s)", (user_id, age, location))
            connection.commit()
            dropdown.options = list(user_id_to_index.keys()) + ['New User', user_id]
            dropdown.value = user_id
            rec_books = get_recommendations(user_id, genre=selected_genre)
            recommendations_output.object = rec_books.to_markdown()
        cursor.close()
        connection.close()
    except mysql.connector.Error as e:
        recommendations_output.object = f":x: Database Error: {e}"
dropdown.param.watch(update_recommendations, 'value')
genre_dropdown.param.watch(update_recommendations, 'value')
save_button.on_click(save_new_user)
layout = pn.Column(
    pn.Row(dropdown, genre_dropdown),
    pn.Row(user_id_input, age_input, location_input, save_button),
    recommendations_output
)
layout.servable()

:white_check_mark: Connected to MySQL Database


C:\Users\SREE GANESHA\AppData\Local\Temp\ipykernel_3880\2670470937.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, connection)


Column
    [0] Row
        [0] Select(options=[8, 9, 12, 14, ...], value=8)
        [1] Select(options=['All', 'Travel', ...], value='All')
    [1] Row
        [0] IntInput(name='User ID')
        [1] IntInput(name='Age')
        [2] TextInput(name='Location')
        [3] Button(button_type='success', name='Save New User')
    [2] Markdown(None)

### collabrative filtering 

In [31]:
import mysql.connector
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import panel as pn

pn.extension()

# --------------------------
# Database Connection
# --------------------------
def connect_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="genrebookrecommendation"
    )

connection = connect_db()
cursor = connection.cursor(dictionary=True)

print("✅ Connected to MySQL Database")

# --------------------------
# Fetch Data from Database
# --------------------------
def fetch_data(query):
    with connection.cursor(dictionary=True) as cursor:
        cursor.execute(query)
        result = cursor.fetchall()
    return pd.DataFrame(result)

books_df = fetch_data("SELECT ISBN, Book_Title, Book_Author, Genre FROM books")
users_df = fetch_data("SELECT User_ID FROM users")
ratings_df = fetch_data("SELECT User_ID, ISBN, Book_Rating FROM ratings")

if ratings_df.empty:
    print("❌ Error: No ratings found in the database.")
else:
    # Create user-item matrix
    user_item_matrix = ratings_df.pivot(index='User_ID', columns='ISBN', values='Book_Rating').fillna(0)
    matrix_sparse = csr_matrix(user_item_matrix)

    # Train KNN model
    model = NearestNeighbors(metric='cosine', algorithm='brute')
    model.fit(matrix_sparse)

    user_id_to_index = {uid: i for i, uid in enumerate(user_item_matrix.index)}
    user_ids = sorted(user_id_to_index.keys())

# --------------------------
# Get Related Genres Dynamically
# --------------------------
def get_related_genres(selected_genre):
    """Find genres that have users who rated books in the selected genre."""
    related_books = books_df[books_df["Genre"] == selected_genre]
    if related_books.empty:
        return [selected_genre]

    # Get users who rated books in the selected genre
    users_with_genre = ratings_df[ratings_df["ISBN"].isin(related_books["ISBN"])]["User_ID"].unique()

    # Find books rated by those users in other genres
    related_books_df = ratings_df[ratings_df["User_ID"].isin(users_with_genre)]
    related_genres = books_df[books_df["ISBN"].isin(related_books_df["ISBN"])]["Genre"].unique().tolist()

    return list(set(related_genres + [selected_genre]))  # Ensure the selected genre is included

# --------------------------
# Recommendation Functions
# --------------------------
def get_recommendations(user_idx, selected_genre, top_n=5):
    """Get book recommendations based on similar users and related genres."""
    user_vector = matrix_sparse[user_idx].reshape(1, -1)
    distances, indices = model.kneighbors(user_vector, n_neighbors=top_n + 1)
    similar_users = indices.flatten()[1:]

    related_genres = get_related_genres(selected_genre)

    recommended_books = ratings_df[ratings_df['User_ID'].isin(user_item_matrix.index[similar_users])]
    recommendations = recommended_books.groupby('ISBN').mean().sort_values('Book_Rating', ascending=False).head(20)
    recommendations = recommendations.merge(books_df, on='ISBN', how='left')

    filtered_recommendations = recommendations[recommendations['Genre'].isin(related_genres)]

    # If no books found in related genres, show books from any genre
    if filtered_recommendations.empty:
        return recommendations[['Book_Title', 'Book_Author', 'Book_Rating', 'Genre']].head(5)

    return filtered_recommendations[['Book_Title', 'Book_Author', 'Book_Rating', 'Genre']]

def get_popular_books(selected_genre, top_n=5):
    """Suggest popular books from the selected and related genres."""
    related_genres = get_related_genres(selected_genre)

    popular_books = ratings_df.groupby('ISBN')['Book_Rating'].mean().sort_values(ascending=False).head(50).index
    popular_books_df = books_df[(books_df['ISBN'].isin(popular_books)) & (books_df['Genre'].isin(related_genres))]

    if popular_books_df.empty:
        return books_df.sample(n=min(5, len(books_df)))[['Book_Title', 'Book_Author', 'Genre']]

    return popular_books_df[['Book_Title', 'Book_Author', 'Genre']].head(5)

# --------------------------
# Panel UI
# --------------------------
dropdown_user = pn.widgets.Select(name="Select User", options=user_ids + ["New User"])
dropdown_genre = pn.widgets.Select(name="Select Genre", options=books_df["Genre"].unique().tolist())

# Widgets for new user
new_user_id = pn.widgets.IntInput(name="User ID")
new_user_age = pn.widgets.IntInput(name="Age")
new_user_location = pn.widgets.TextInput(name="Location")
save_button = pn.widgets.Button(name="Save New User", button_type="success")

output_pane = pn.pane.Markdown()

def update_recommendations(event):
    output_pane.object = ""  # Clear output
    selected_user_id = dropdown_user.value
    selected_genre = dropdown_genre.value

    if selected_user_id == "New User":
        return

    if isinstance(selected_user_id, int) and selected_user_id in user_id_to_index:
        user_idx = user_id_to_index[selected_user_id]
        rec_books = get_recommendations(user_idx, selected_genre)

        if rec_books.empty:
            rec_books = get_popular_books(selected_genre)

        output_pane.object = f"### 📚 Recommendations for User {selected_user_id} (Genre: {selected_genre}):\n{rec_books.to_markdown()}"

def save_new_user(event):
    output_pane.object = ""
    user_id = new_user_id.value
    age = new_user_age.value
    location = new_user_location.value
    selected_genre = dropdown_genre.value

    if not user_id or not age or not location:
        output_pane.object = "❌ Please fill in all fields!"
        return

    try:
        connection = connect_db()
        cursor = connection.cursor()

        cursor.execute("SELECT User_ID FROM users WHERE User_ID = %s", (user_id,))
        existing_user = cursor.fetchone()

        if existing_user:
            output_pane.object = f"⚠️ User {user_id} already exists! Showing recommendations."
        else:
            cursor.execute("INSERT INTO users (User_ID, Age, Location) VALUES (%s, %s, %s)", (user_id, age, location))
            connection.commit()

            output_pane.object = f"✅ New user {user_id} added successfully!"
            user_ids.append(user_id)
            dropdown_user.options = user_ids + ["New User"]

        dropdown_user.value = user_id
        rec_books = get_popular_books(selected_genre)
        output_pane.object += f"\n### 📚 Recommendations for New User {user_id} (Genre: {selected_genre}):\n{rec_books.to_markdown()}"

        cursor.close()
        connection.close()

    except mysql.connector.Error as e:
        output_pane.object = f"❌ Error: {e}"

# Attach event listeners
dropdown_user.param.watch(update_recommendations, 'value')
save_button.on_click(save_new_user)

# Layout
app = pn.Column(
    pn.pane.Markdown("# 📖 Book Recommendation System"),
    dropdown_user,
    dropdown_genre,
    pn.Row(new_user_id, new_user_age, new_user_location),
    save_button,
    output_pane
)

# Display in Jupyter Notebook
app.servable()


✅ Connected to MySQL Database


Column
    [0] Markdown(str)
    [1] Select(options=[8, 9, 12, 14, ...], value=8)
    [2] Select(options=['Travel', 'Religion', ...], value='Travel')
    [3] Row
        [0] IntInput(name='User ID')
        [1] IntInput(name='Age')
        [2] TextInput(name='Location')
    [4] Button(button_type='success', name='Save New User')
    [5] Markdown(None)

### popularity based filtering 

In [3]:
import mysql.connector
import pandas as pd
import panel as pn

# Make sure Panel extension is loaded
pn.extension()

# --------------------------
# Database Connection
# --------------------------
def connect_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="genrebookrecommendation"
    )
connection = connect_db()
cursor = connection.cursor()
print("\u2705 Connected to MySQL Database")

# --------------------------
# Fetch Data from Database
# --------------------------
def fetch_data(query):
    return pd.read_sql(query, connection)

books_df = fetch_data("SELECT ISBN, Book_Title, Book_Author, Genre FROM books")
ratings_df = fetch_data("SELECT User_ID, ISBN FROM ratings")
users_df = fetch_data("SELECT User_ID, Age, Location FROM users")

# Merge ratings with books
ratings_books_df = ratings_df.merge(books_df, on='ISBN', how='inner')

# Count number of ratings per book
popular_books = ratings_books_df.groupby(['Book_Title', 'Book_Author', 'Genre'])['ISBN'].count().reset_index()
popular_books.rename(columns={'ISBN': 'rating_count'}, inplace=True)

# Get unique genres
unique_genres = sorted(books_df['Genre'].dropna().unique().tolist())

# Get unique users + "Other" option
user_ids = sorted(users_df['User_ID'].astype(str).tolist()) + ["Other"]

# --------------------------
# Get Top 3 Books Per Genre
# --------------------------
def get_top_books_per_genre():
    top_books_per_genre = {}
    for genre in unique_genres:
        genre_books = popular_books[popular_books['Genre'] == genre].sort_values(by='rating_count', ascending=False).head(3)
        if not genre_books.empty:
            top_books_per_genre[genre] = genre_books
    return top_books_per_genre

# --------------------------
# Interactive UI with Panel
# --------------------------
user_id_dropdown = pn.widgets.Select(name="Select User", options=user_ids)
new_user_id_input = pn.widgets.TextInput(name="New User ID", visible=False)
age_input = pn.widgets.IntInput(name="Age", visible=False)
location_input = pn.widgets.TextInput(name="Location", visible=False)
save_button = pn.widgets.Button(name="Save User", button_type="success", visible=False)
recommend_button = pn.widgets.Button(name="Get Recommendations", button_type="primary")
output = pn.pane.Markdown("", width=600)

# Function to check if user exists
def check_user_exists(user_id):
    return user_id in users_df['User_ID'].astype(str).values

# Function to update UI when user selection changes
def on_user_selected(event):
    selected_user = event.new
    if selected_user == "Other":
        new_user_id_input.visible = True
        age_input.visible = True
        location_input.visible = True
        save_button.visible = True
    else:
        new_user_id_input.visible = False
        age_input.visible = False
        location_input.visible = False
        save_button.visible = False
        get_recommendations()

user_id_dropdown.param.watch(on_user_selected, 'value')

# Function to save new user
def save_new_user(event):
    user_id = new_user_id_input.value.strip()
    age = age_input.value
    location = location_input.value.strip()
    if not user_id or not age or not location:
        output.object = "\u274C Please fill in all fields!"
        return
    if check_user_exists(user_id):
        output.object = f"\u274C User ID '{user_id}' already exists! Choose a different one."
        return
    try:
        connection = connect_db()
        cursor = connection.cursor()
        insert_query = "INSERT INTO users (User_ID, Age, Location) VALUES (%s, %s, %s)"
        cursor.execute(insert_query, (user_id, age, location))
        connection.commit()
        cursor.close()
        connection.close()
        output.object = f"\u2705 New user '{user_id}' added successfully!"
        user_ids.insert(-1, user_id)  # Add before "Other"
        user_id_dropdown.options = user_ids
        user_id_dropdown.value = user_id
        get_recommendations()
    except mysql.connector.Error as e:
        output.object = f"\u274C Error: {e}"

save_button.on_click(save_new_user)

# Function to get recommendations
def get_recommendations(event=None):
    selected_user = user_id_dropdown.value
    if selected_user == "Other":
        output.object = ":warning: Please enter new user details first!"
        return
    top_books_per_genre = get_top_books_per_genre()
    if top_books_per_genre:
        recommendations = "## :books: Top 3 Books per Genre:\n"
        for genre, books in top_books_per_genre.items():
            recommendations += f"\n### :small_blue_diamond: Popular Books for {genre}:\n"
            recommendations += books.to_markdown(index=False) + "\n"
        output.object = recommendations
    else:
        output.object = "\u274C No books found."

recommend_button.on_click(get_recommendations)

# Layout UI
pn.Column(
    user_id_dropdown,
    new_user_id_input,
    age_input,
    location_input,
    save_button,
    recommend_button,
    output
).servable()


✅ Connected to MySQL Database


C:\Users\SREE GANESHA\AppData\Local\Temp\ipykernel_3140\1273428515.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, connection)


Column
    [0] Select(options=['1', '10', '102', ...], value='1')
    [1] TextInput(name='New User ID', visible=False)
    [2] IntInput(name='Age', visible=False)
    [3] TextInput(name='Location', visible=False)
    [4] Button(button_type='success', name='Save User', visible=False)
    [5] Button(button_type='primary', name='Get Recommendations')
    [6] Markdown(str, width=600)

### READER MOOD BASED 

In [29]:
import panel as pn
import pandas as pd

# Enable Panel for Jupyter
pn.extension()

# Load data
books = pd.read_csv("./books.csv")
ratings = pd.read_csv("./rating.csv")
users = pd.read_csv("./users.csv")

# User-ID dropdown (includes "New User")
user_ids = list(users["User-ID"].unique())
user_ids.insert(0, "New User")
user_select = pn.widgets.Select(name="Select User", options=user_ids)

# New user input fields (initially hidden)
new_user_id = pn.widgets.TextInput(name="Enter User ID", placeholder="e.g., 10001")
new_user_name = pn.widgets.TextInput(name="Enter Name", placeholder="e.g., John Doe")
new_user_age = pn.widgets.IntInput(name="Enter Age", placeholder="e.g., 25", step=1)
new_user_location = pn.widgets.TextInput(name="Enter Location", placeholder="e.g., Hyderabad")

new_user_form = pn.Column(new_user_id, new_user_name, new_user_age, new_user_location, visible=False)

# Mood mapping
mood_mapping = {
    "Cheerful": "Happy", "Moody": "Sad", "Calm": "Calm", "Tired": "Tired",
    "Laugh and have fun": "Happy", "Reflect and relax": "Sad", "Go on an adventure": "Happy", "Read something emotional": "Sad",
    "High energy": "Happy", "Low energy": "Sad", "Balanced energy": "Calm", "Very low energy": "Tired",
    "Happy, excited": "Happy", "Sad, nostalgic": "Sad", "Peaceful, content": "Calm", "Motivated, adventurous": "Happy",
    "Action-packed stories": "Happy", "Deep emotional stories": "Sad", "Relaxing and slow-paced": "Calm", "Short and light reads": "Tired"
}

mood_to_keywords = {
    "Happy": ["adventure", "fun", "joy", "excitement"],
    "Sad": ["nostalgic", "reflective", "deep"],
    "Calm": ["peaceful", "relaxing", "soothing"],
    "Tired": ["light", "easy", "gentle"]
}

def recommend_books(mood, top_n=5):
    """Recommend books based on the predicted mood."""
    keywords = mood_to_keywords.get(mood, [])
    if "Book-Title" not in books.columns:
        return pd.DataFrame({"Error": ["Book-Title column missing in dataset"]})
    
    recommended_books = books[books['Genre'].str.contains('|'.join(keywords), case=False, na=False)]
    if recommended_books.empty:
        recommended_books = books.sample(n=min(top_n, len(books)))  # Random books if no exact match
    
    return recommended_books[['Book-Title', 'Book-Author', 'Genre']].head(top_n)

# Create dropdown widgets for 5 mood-related questions
q1 = pn.widgets.Select(name="How do you feel today?", options=["Cheerful", "Moody", "Calm", "Tired"])
q2 = pn.widgets.Select(name="What do you want to do right now?", options=["Laugh and have fun", "Reflect and relax", "Go on an adventure", "Read something emotional"])
q3 = pn.widgets.Select(name="How would you describe your energy level?", options=["High energy", "Low energy", "Balanced energy", "Very low energy"])
q4 = pn.widgets.Select(name="What kind of emotions do you feel today?", options=["Happy, excited", "Sad, nostalgic", "Peaceful, content", "Motivated, adventurous"])
q5 = pn.widgets.Select(name="What type of books do you prefer?", options=["Action-packed stories", "Deep emotional stories", "Relaxing and slow-paced", "Short and light reads"])

output_area = pn.pane.DataFrame(width=600)  # Placeholder for output
button = pn.widgets.Button(name="Get Book Recommendations", button_type="primary")

# Function to update recommendations
def update_recommendations(event):
    selected_user = user_select.value

    # If "New User" is selected, store new user details
    if selected_user == "New User":
        if not new_user_id.value or not new_user_name.value or not new_user_age.value or not new_user_location.value:
            output_area.object = pd.DataFrame({"Error": ["Please fill in all details for new user."]})
            return
        
        selected_user = new_user_id.value
        new_user_data = pd.DataFrame({
            "User-ID": [selected_user],
            "Name": [new_user_name.value],
            "Age": [new_user_age.value],
            "Location": [new_user_location.value]
        })
        print("New user registered:", new_user_data)  # Log new user (can be stored in DB)

    # Get user answers
    mood1 = mood_mapping[q1.value]
    mood2 = mood_mapping[q2.value]
    mood3 = mood_mapping[q3.value]
    mood4 = mood_mapping[q4.value]
    mood5 = mood_mapping[q5.value]
    
    # Determine the most frequent mood
    mood_counts = {"Happy": 0, "Sad": 0, "Calm": 0, "Tired": 0}
    for mood in [mood1, mood2, mood3, mood4, mood5]:
        mood_counts[mood] += 1
    
    # Final mood is the most chosen mood
    final_mood = max(mood_counts, key=mood_counts.get)
    
    # Get recommendations
    recommendations = recommend_books(final_mood)
    
    # Update the output
    output_area.object = recommendations

button.on_click(update_recommendations)

# Function to show/hide new user fields
def update_user_fields(event):
    new_user_form.visible = user_select.value == "New User"

user_select.param.watch(update_user_fields, "value")

# Display UI
dashboard = pn.Column(
    "# Mood-Based Book Recommendation",
    user_select,  # User selection dropdown
    new_user_form,  # New user details (hidden initially)
    q1, q2, q3, q4, q5,
    button,
    output_area
)

dashboard.servable()
dashboard


New user registered: User-ID Name Age Location 
0 45454555 dhathri 19 hyderabad

Column
    [0] Markdown(str)
    [1] Select(options=['New User', 2, ...], value='New User')
    [2] Column(visible=False)
        [0] TextInput(name='Enter User ID', placeholder='e.g., 10001')
        [1] TextInput(name='Enter Name', placeholder='e.g., John Doe')
        [2] IntInput(name='Enter Age', placeholder='e.g., 25')
        [3] TextInput(name='Enter Location', placeholder='e.g., Hyderabad')
    [3] Select(name='How do you feel today?', options=['Cheerful', 'Moody', ...], value='Cheerful')
    [4] Select(name='What do you w..., options=['Laugh and have fun', ...], value='Laugh and have fun')
    [5] Select(name='How would you d..., options=['High energy', ...], value='High energy')
    [6] Select(name='What kind of e..., options=['Happy, excited', ...], value='Happy, excited')
    [7] Select(name='What type of b..., options=['Action-packed stories', ...], value='Action-packed stories')
    [8] Button(button_type='primary', name='Get Book Recommendations'...)
    [9] DataFrame(None, width=600)

### SAVING MODELS

In [34]:
import pickle
import os
from sklearn.neighbors import NearestNeighbors  # Import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer  # Import TfidfVectorizer
# Ensure the models directory exists
os.makedirs("../models", exist_ok=True)
# Example models (Replace these with your actual models)
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
tfidf_vectorizer = TfidfVectorizer()
# Save models
pickle.dump(knn_model, open("../models/knn_model.sav", "wb"))
pickle.dump(tfidf_vectorizer, open("../models/tfidf_vectorizer.sav", "wb"))
print("Models saved successfully in the 'models' folder.")

Models saved successfully in the 'models' folder.
